# Predicting Loan Default: Part 3b

**Running Best Model on Test**

In [23]:
# Importing Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, RocCurveDisplay, log_loss, f1_score
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, train_test_split

from xgboost import XGBClassifier

import re

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

In [13]:
# read in dataset
train = pd.read_csv('data/train_model_a.csv')

In [14]:
train = train.set_index('id')

In [15]:
from sklearn.preprocessing import LabelEncoder
for column in train.columns:
    if train[column].dtype==np.number:
        continue
    train[column]=LabelEncoder().fit_transform(train[column])

C:\Users\khogu\AppData\Local\Temp\ipykernel_35360\3845078900.py:3: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if train[column].dtype==np.number:
C:\Users\khogu\AppData\Local\Temp\ipykernel_35360\3845078900.py:3: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if train[column].dtype==np.number:
C:\Users\khogu\AppData\Local\Temp\ipykernel_35360\3845078900.py:3: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if train[column].dtype==np.number:
C:\Users\khogu\AppData\Local\Temp\ipykernel_35360\3845078900.py:3: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if train[column].dt

In [16]:
# create a new test dataset using 20% of the train data
train, test = train_test_split(train, test_size=0.2, stratify=train['loan_status'])

print(train['loan_status'].value_counts())
print(test['loan_status'].value_counts())

0    48977
1     4993
Name: loan_status, dtype: int64
0    12245
1     1248
Name: loan_status, dtype: int64


**Running the best model on test data**

Best Params: {'smote__sampling_strategy': 'minority', 'xgb__learning_rate': 0.14, 'xgb__max_depth': 4, 'xgb__n_estimators': 150}
Best Score: 0.5161713545211244

In [17]:
# Train test split
y = train['loan_status']
X = train.drop('loan_status', axis=1)

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state = 42, stratify =y)

In [18]:
# define X and y for test

X_test = test.drop('loan_status', axis = 1)
y_test = test['loan_status']

In [19]:
# Fit XGBoost with best parameters
xgb = XGBClassifier(random_state=42, learning_rate=0.14, max_depth=4, n_estimators=150)
smote = SMOTE(random_state=42, sampling_strategy='minority')
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
xgb.fit(X_train_resampled, y_train_resampled)

y_pred = xgb.predict(X_test)

In [27]:
# Obtain predicted probabilities for test data
y_pred_proba = xgb.predict_proba(X_test)

# Calculate log loss score for test data
logloss = log_loss(y_test, y_pred_proba)

# Print the log loss score
print("Log Loss Score: {:.5f}".format(logloss))
print("Accuracy Score:", round(accuracy_score(y_test, xgb.predict(X_test)), 4))
print("F1 Score:", round(f1_score(y_test, xgb.predict(X_test)), 4))

Log Loss Score: 0.52156
Accuracy Score: 0.7605
F1 Score: 0.1246


**Conclusion**

Just as a benchmark, the Log Loss score for the top performer in this competition was between 0.34 to 0.35.

The process of feature engineering when dealing with a dataset like this is important in getting better scores.

Binning features and making them categorical seems to work especially well with Random Forest Classifier and XGBoost.

**Recommendations**

The model is not yet the finished product to be deployed, but can be used as a base to continue improving.

Feature engineering based on the feature importance from the best model will play a big role in the next steps in the next phase of the project which I will continue to work on over the next few weeks.

Obtaining the right data from customers to be used as features in the model besides what was provided might prove more useful than tweaking existing data.

In order to get the model good enough to deploy, a few things need to be accomplished.

1. Feature engineering and continued tuning to get better Accuracy and F1 scores
2. Reduction of features such that the deployment feature ranges are easily obtainable by bank clients and the bank.
3. Model needs to run fast in order to be deployed for customer use.